Following this course: https://github.com/louthy/language-ext/wiki/Thinking-Functionally    

In [ ]:
// quick setup
#r "nuget:Shouldly"
#r "nuget:LanguageExt.Core"
using Shouldly;
using LanguageExt;
using static LanguageExt.Prelude;

Installed Packages LanguageExt.Core, 4.2.9 Shouldly, 4.0.3

### Sarting simple
`AddOne :: int -> int`    
A function "maps" a value in the "domain" to a value in the "range". There is no "computation".

In [ ]:
int AddOne(int x) => x + 1;
return AddOne(5);

6

#### Important notes:    
`x` is not something that can change once we pass 5 in.    
This is not assignment - What is happening here is "binding".    
Once a domain value is "bound" is cannot be changed.     
`x` is a placeholder that can be referred to later, not changed.    
**There are no "variables", only values**

_side note, yes x is a variable being assigned a value - but it is forbidden 'functionaly'_

### Function Values
`Func<domain, range>`    

Similarly - the name `AddOne` is just a "binding" to "the function that adds one to its input"    
"Every time you see the name `AddOne`, replace it with the function that adds one to its input".    

For example, we can "bind" `AddOne` to a new name.    
`PlusOne` and `AddOne` are "bound" to the same value, which is the function that adds one to its input.    

In [ ]:
Func<int, int> PlusOne = AddOne;
return (PlusOne(5), AddOne(5));

Item1,Item2
6,6


### Simple Value
Imagine an operation that always returns `5` - this is a constant.    
in "C#" you can easily define that with the following: `const int C = 5;` or `static int C => 5; //Func<int>`.      
simple values do not need to be evaluated once "bound" - This is the subtle difference between them and function values, otherwise they are both values of Func.

With this, `5` and `Five` are both just values that can be passed around. - This is a key part of thinking functionaly: functions are values that can be passed around to other functions.

A constant function for `5` would be defined as:    
`Func<Unit, int>`   

In [ ]:
const int C = 5;
//or
static int C_alt => 5;
//or
Func<Unit, int> Five = _ => 5;
//or
static int Five_alt(Unit _) => 5;

return (AddOne(C), AddOne(C_alt), AddOne(Five(unit)), AddOne(Five_alt(unit)));

// the presence of "Unit" is a bit annoying, this gets cleaned up later on.

Item1,Item2,Item3,Item4
6,6,6,6


### "Objects" vs "Values"

Values:
- A member of a domain. The domain of ints, strings, functions that map ints to strings, etc...    
- They are immutable
- They have no behaviour

Objects: 
- Encapsulation of data with associated behaviour
- Mutable

Avoid using "object" for standard values for functional C#.    
It should only be used to refer to true classes, or values that expose member functions.

### How types work with functions

Type notation: `domain -> range` == (C#) `Func<domain, range>`

"Higher-Order functions": A function that takes a function as a paramater, or returns another function.    
Think of LINQ `Select` or `Where`.

Example:

In [ ]:
static int EvalWith5ThenAdd2(Func<int, int> fn) => 
    fn(5) + 2;

return EvalWith5ThenAdd2(AddOne);

8

In this function the domain is `(int -> int)` and the range is `int`.    
This means the input is a function that maps ints to ints, and the output is just an int.

In [ ]:
Func<int, int> TimesThree = x => x * 3;

return EvalWith5ThenAdd2(TimesThree);

17

Now, lets return a function rather than use one as input.    
`int -> (int -> int)`

In [ ]:
static Func<int, int> AdderGenerator(int numberToAdd) => 
    x =>  x + numberToAdd;

//or

Func<int, Func<int,int>> AdderGenerator_alt = numberToAdd => 
    x => x + numberToAdd;

// create new adders
var add1 = AdderGenerator(1);
var add2 = AdderGenerator_alt(2);

return (add1(1), add2(1));

Item1,Item2
2,3


### void?

How do we deal with something like:
```csharp
void PrintInt(int x)
{
    Console.WriteLine(x);
}
```
We cannot have `int -> void` since void is the absence of a value. But our function must have a range to map to.  Every function must have some output to map to.    
`Unit` to the rescue - it is like a `bool` which has two possible values `true`, `false`, though it only has one possible value `unit`.


In Funtional langauges the `Unit` type is often reffered to as `()` - however in C# this is a method invocation with no inputs.    
Since declaring this concept in c# fully is a bit verbose as we saw earlier with `Five` it is often omitted as an input.    

Example:

In [ ]:
Unit WhatIsThis(Unit _) => unit;
// or better yet - which captures a bit of that functional syntax by omitting the unit input
Unit WhatIsThis() => unit;

// doesn't compile because we must return a unit
/*
Unit DoSomething() => 1 + 1; 
*/
Unit DoSomething() => ignore(1 + 1); // ignore is a wrapper to satisfy the c# compiler

#### Other useful types

In [ ]:
var tuples = ("Hello", 1);
Lst<int> list = List(1,2,3); // immutable, fixed size
Arr<int> arr = Array(1,2,3); // immutable, fixed size
Seq<int> seq = Seq(Range(1,10)); // similar to IEnumerable, but not infinite

Option<int> someInt = Some(1); // options wrap values that may be missing
Option<int> noInt = None;